# FastANI tree

In [ ]:
#@markdown  #Install dependencies
!pip install biopython 2> /dev/null 1> logs.txt

import os
if not os.path.exists('./datasets'):
  print('Instalando Datasets')
  !wget https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/v2/linux-amd64/datasets 2> /dev/null 1>> logs.txt
  !chmod +x datasets
else:
  print('Datasets already installed...')

if not os.path.exists('/content/fastANI'):
  print(f'Instalando FastAni')
  !wget https://github.com/ParBLiSS/FastANI/releases/download/v1.34/fastANI-linux64-v1.34.zip 2> /dev/null 1>> logs.txt
  !unzip fastANI-linux64-v1.34.zip 2> /dev/null 1>> logs.txt

print('Instalando phylip')
!sudo apt-get install phylip 2> /dev/null 1>> logs.txt

In [ ]:
#@markdown # Download genomes from NCBI
#@markdown **Input**: a _genus_ _species_ name to a space separated list of NCBI assembly accession numbers
#get accessions or sp
#@markdown **Set "get_sp" to ON** to download all the genomes from NCBI, otherwise download by accession number lsit.
get_sp = True #@param{type:'boolean'}

get_species_genomes = '' #@param {type: 'string'}

#assembly level filter
assembly_level = True #@param {type: 'boolean'}
if assembly_level:
  assembly_level_flag = '--assembly-level complete'
else:
  assembly_level_flag = ""

#Annotated
annotated = True #@param {type: 'boolean'}
if annotated:
  annotated_flag = '--annotated'
else:
  annotated_flag = ""

#exclude-atypical
exclude_atypical = True #@param {type: 'boolean'}
if exclude_atypical:
  exclude_atypical_flag = '--exclude-atypical'
else:
  exclude_atypical_flag = ""

#refseq only
refseq_only = True #@param {type: 'boolean'}
if refseq_only:
  refseq_only_flag = '--assembly-source RefSeq'
else:
  refseq_only_flag = ""


#@markdown ### Use if you want to download an accession list
genome_accessions = '' #@param {type: 'string'}
genome_accessions_list = genome_accessions.split()

import os, glob

try:
  if get_sp:
    try:
      !./datasets download genome taxon "{get_species_genomes}" {assembly_level_flag} {exclude_atypical_flag } {refseq_only_flag}
      !unzip ncbi_dataset
      genome_list = glob.glob('/content/ncbi_dataset/data/*/**/*.fna', recursive=True)
      base_names = {os.path.basename(i).split('.')[0]:i for i in genome_list}
      acc = [(i.split('_')[0],i.split('_')[1].split('.')[0]) for i in base_names.keys()]
      ref_seqs = {i[1]:'_'.join(i) for i in acc if i[0] == 'GCF'}
      genomes = set([ '_'.join(i) if i[1] not in ref_seqs else ref_seqs[i[1]] for i in acc])
      genome_files = [ base_names[i] for i in genomes]
      #os.remove(f'/content/ncbi_dataset.zip')
    except:
      print(f'Error: {get_species_genomes} not found')
  else:
    %mkdir /content/genomes
    genome_files = []
    for acc in genome_accessions_list:
      if not os.path.exists(f'/content/genomes/{acc}.gb'):
        print(f'Downloading {acc} genome...')
        try:
          #!./datasets download genome accession {acc} --include gbff --filename {acc}.zip >/dev/null 2>&1
          !./datasets download genome accession {acc} --filename {acc}.zip >/dev/null 2>&1
          #!unzip -o -j {acc}.zip ncbi_dataset/data/{acc}/genomic.gbff -d ./ >/dev/null 2>&1
          !unzip -o -j {acc}.zip ncbi_dataset/data/{acc}/*.fna -d ./ >/dev/null 2>&1
          os.remove(f'{acc}.zip')
          gfile = glob.glob(f'{acc}*.fna')[0]
          !mv {gfile} /content/genomes/{acc}.fna >/dev/null 2>&1
          genome_files.append(f'/content/genomes/{acc}.fna')
        except:
          print(f'Error: {acc} not found')
except:
  print('Error: Something went wrong')



In [ ]:
#@markdown #Extract info from Genbank file

In [ ]:
#@markdown ###Zip and download genomes to local computer
import os
import shutil
from google.colab import files
if get_sp:
  for gnome in genome_files:
    shutil.copy(gnome, os.path.join('/content/',os.path.join('/content/genomes/',os.path.basename(gnome))))

!zip -r /content/genomes.zip /content/genomes
files.download("/content/genomes.zip")

#Make FastANI analysis
Slow in Colab with only two threads... for 30 genomes max...  
This makes an all vs all comparison.

In [ ]:
#@markdown ### Run FastANI ALL vs ALL
#@markdown Warning: No ANI output is reported for a genome pair if ANI value is much below 80%.
import os
QUERY_LIST = 'query_list.txt'
#ncpu = !nproc
#ncpu = int(ncpu[0])
ncpu = os.cpu_count()
with open(QUERY_LIST,'w') as f:
  for genome in genome_files:
    f.write(f'{genome}\n')
!./fastANI --ql {QUERY_LIST} --rl {QUERY_LIST} -o fastani.out --matrix -t {ncpu}


In [ ]:
#@markdown ### Run FastANI Ref vs ALL
#@markdown Warning: No ANI output is reported for a genome pair if ANI value is much below 80%.
#@markdown Pleas input the REF_SEQ Assembly accession number
import os

def download_REF_genome(acc):
  try:
      !./datasets download genome accession {acc} --filename {acc}.zip >/dev/null 2>&1
      !unzip -o -j {acc}.zip ncbi_dataset/data/{acc}/*.fna -d ./ >/dev/null 2>&1
      os.remove(f'{acc}.zip')
      gfile = glob.glob(f'{acc}*.fna')[0]
      if not os.path.exists('/content/genomes/'):
        os.mkdir('/content/genomes/')
      !mv {gfile} /content/genomes/{acc}.fna >/dev/null 2>&1
      return(f'/content/genomes/{acc}.fna')
  except:
    print(f'Error: {acc} not found')
    return(False)


REF = '' #@param {type:"string"}
ref_genome = download_REF_genome(REF)
if ref_genome:
  print(f"Using {ref_genome} file as reference genome...")
  QUERY_LIST = 'query_list.txt'
  ncpu = os.cpu_count()
  with open(QUERY_LIST,'w') as f:
    for genome in genome_files:
      f.write(f'{genome}\n')
  !./fastANI -q {ref_genome} --rl {QUERY_LIST} -o fastani.out -t {ncpu}
else:
  print('Error: Something went wrong...')

In [ ]:
#@markdown Process the results and select by ANI difference with the reference an specifed number of genomes to work with.

#@markdown To download the selected genomes check the box:

Download_genomes = True #@param {type:"boolean"}

import pandas as pd
ref_all_ANI = pd.read_table('/content/fastani.out', sep='\t', names=['Ref','Genome','ANI','count of bidirectional fragment mappings','total query fragments'], header = None)
ref_all_ANI.sort_values(['ANI'], ascending=False)
ref_all_ANI['Alignment fraction'] = ref_all_ANI['count of bidirectional fragment mappings']/ref_all_ANI['total query fragments']
alignment_fraction_filtering = True #@param {type:"boolean"}
alignment_fraction = 0.8 #@param {type:"number"}
if alignment_fraction_filtering:
  ref_all_ANI = ref_all_ANI[ref_all_ANI['Alignment fraction'] >= alignment_fraction]

numer_of_Sequences = 50 #@param {type:'integer'}
one_every = int(len(ref_all_ANI)/numer_of_Sequences)
if one_every == 0:
  print(f'Error, less than {numer_of_Sequences} sequences...')
else:
  to_analyze = []
  ref = os.path.basename(os.path.splitext(ref_genome)[0])
  import re
  ref = re.sub('GCA_','GCF_',ref)
  to_analyze.append(ref)
  for i in range(0,len(ref_all_ANI),one_every):
    genomeACC = '_'.join(os.path.splitext(os.path.basename(ref_all_ANI.iloc[i,1]))[0].split('_')[0:2])
    if not genomeACC in to_analyze:
      to_analyze.append(genomeACC)


In [ ]:
#@markdown Download selected genomes
if not os.path.exists('/content/to_dl'):
  %mkdir /content/to_dl

genome_files = []
for acc in to_analyze:
  print(f'Downloading {acc} genome...')
  try:
    !./datasets download genome accession {acc} --include gbff --filename {acc}.zip >/dev/null 2>&1
    !unzip -o -j {acc}.zip ncbi_dataset/data/{acc}/genomic.gbff -d ./ >/dev/null 2>&1
    os.remove(f'{acc}.zip')
    !mv genomic.gbff /content/to_dl/{acc}.gbff >/dev/null 2>&1
    genome_files.append(f'/content/to_dl/{acc}.fna')
  except:
    print(f'Error: {acc} not found')

import shutil
import os
from google.colab import files
print(f'A total of {len(to_analyze)} genomes were selected.')

with open('selected_genomes.csv','w') as f:
  f.write(f'Accession number\n')
  for a in to_analyze:
    f.write(f'{a}\n')
print('The selected accession numbers are on selected_genomes.csv file...')

if Download_genomes:
  print('Downloading genomes in zip file...')
  !zip -r /content/genomes.zip /content/to_dl
  files.download("/content/genomes.zip")



##Make FastANI UPGMA tree.
From here, the steps are **only for all vs all** comparisons. Requiere the Similarity ANI matrix.

In [ ]:
#@markdown ### Convert to distance matrix
#@markdown  FastANI out matrix, default value --> *fastani.out.matrix*
fastani_out_matrix = 'fastani.out.matrix' #@param {type:'string'}

!sed -e 's/\([^/]*\/\)*\([^_]*\)_\([^_]*\)[^\t]*\(.*\)/\2_\3\4/' {fastani_out_matrix} > /content/fastani.matrix

with open('/content/fastani.matrix','r') as f:
  line_dict = {}
  n_seqs = f.readline()
  ids = []
  values = []
  for line in f:
    line = line.strip()
    if (len(line.split('\t')) == 1):
      ids.append(line.split('.')[0])
      values.append([''])
    else:
      id_simil_list =line.split('\t')
      size = len(id_simil_list)
      simil_list = id_simil_list[1:size]
      simil_list = [float(i) for i in simil_list]
      simil_list = [str((100 - i)/100) for i in simil_list]
      ids.append(id_simil_list[0].split('.')[0])
      values.append(simil_list)

#generate dist matrix
with open('dist.matrix','w') as f:
  f.write(f'\t{n_seqs}')
  for k,v in zip(list(range(len(ids))),values):
    nl = '\t'.join(v)
    id = 'seq'+'0'*(10-3-len(str(k)))+str(k)
    line = f'{id}\t{nl}'.strip()+'\n'
    f.write(line)

with open('id_list.txt','w') as f:
  id_dict = {}
  for i,v in enumerate(ids):
    id = 'seq'+'0'*(10-3-len(str(i)))+str(i)
    f.write(f'{id},{v}\n')
    id_dict[id]=v

In [ ]:
#@markdown #Run phylip to make a newick tree file
import os

with open ('input','w') as f:
  f.write('''dist.matrix
N
L
Y''')
if os.path.exists('outfile'):
  os.remove('outfile')
if os.path.exists('outtree'):
  os.remove('outtree')
!phylip neighbor < input > screenout


In [ ]:
#@markdown ##Downloads information from NCBI and creates a table
import pandas as pd
import json

get_species_genomes = '' #@param {type: 'string'}

try:
  json_f = !./datasets summary genome taxon "{get_species_genomes}"
except:
  print('Error.. incorrect Genus and species')

report_dict = json_f[0]
report_dict = json.loads(report_dict)
reports = report_dict["reports"]

#table
ncbi_reports = pd.DataFrame(columns = ['Accession','Name','Strain','TaxID','Assembly level', 'N50', 'Genome length','SP by ANI', 'Check-M Completness', 'Proteins'])
for genome_report in reports:
  accession = genome_report['accession']
  if 'organism' in genome_report:
    if 'organism_name' in genome_report['organism']:
      name = genome_report['organism']['organism_name']
    else:
      name = ''
    if 'infraspecific_names' in genome_report['organism']:
      if 'strain' in genome_report['organism']['infraspecific_names']:
        strain = genome_report['organism']['infraspecific_names']['strain']
      else:
        strain = ''
    if 'tax_id' in genome_report['organism']:
      taxid = genome_report['organism']['tax_id']
    else:
      taxid = ''
  else:
    name = ''
    strain = ''
    taxid = ''
  if 'assembly_info' in genome_report:
    if 'assembly_level' in genome_report['assembly_info']:
      assembly_level = genome_report['assembly_info']['assembly_level']
    else:
      assembly_level = ''
  if 'assembly_stats' in genome_report:
    if 'contig_n50' in genome_report['assembly_stats']:
      n50 = genome_report['assembly_stats']['contig_n50']
    else:
      n50 = ''
    if 'total_sequence_length' in genome_report['assembly_stats']:
      genome_length = genome_report['assembly_stats']['total_sequence_length']
    else:
      genome_length = ''
  if 'average_nucleotide_identity' in genome_report:
    if 'submitted_ani_match' in genome_report['average_nucleotide_identity']:
      if 'organism_name' in genome_report['average_nucleotide_identity']['submitted_ani_match']:
        spbyani = genome_report['average_nucleotide_identity']['submitted_ani_match']['organism_name']
      else:
        spbyani = ''
  if 'checkm_info' in genome_report:
    if 'completeness' in genome_report['checkm_info']:
      checkm = genome_report['checkm_info']['completeness']
    else:
      checkm = ''
  if 'annotation_info' in genome_report:
    if 'stats' in genome_report['annotation_info']:
      if 'gene_counts' in genome_report['annotation_info']['stats']:
        if 'protein_coding' in genome_report['annotation_info']['stats']['gene_counts']:
          proteins = genome_report['annotation_info']['stats']['gene_counts']['protein_coding']
        else:
          proteins = ''
  rep_tab = pd.DataFrame.from_records([{'Accession':accession, 'Name': name, 'Strain': strain, 'TaxID':taxid, 'Assembly level':assembly_level, 'N50':n50, 'Genome length':genome_length, 'SP by ANI':spbyani, 'Check-M Completness': checkm, 'Proteins': proteins}])
  ncbi_reports = pd.concat([rep_tab,ncbi_reports])

In [ ]:
#@markdown ##Prints NCBI summary table and exports it to csv format
print(ncbi_reports)
ncbi_reports.to_csv('NCBI_info.csv')

In [ ]:
#@markdown ### Reads NCBI table and generates id equivalences
import pandas as pd

id_2_dict = {}
for i,row in ncbi_reports.iterrows():
  acc = row['Accession'].split('.')[0]
  if acc in id_dict.values():
    id = [k for k,v in id_dict.items() if v == acc][0]
    abb_gen = row['Name'].split()[0][0]
    species = row['Name'].split()[1]
    strain = row['Strain']
    name = f'{abb_gen}. {species} {strain} [{acc}]'
    id_2_dict[id] = name

print(id_2_dict)

In [ ]:
#@markdown #### Correct the ids in tree
import pandas as pd
from Bio import Phylo
tree = Phylo.read('/content/outtree','newick')
for leave in tree.get_terminals():
  if leave.name in id_2_dict.keys():
    leave.name = id_2_dict[leave.name]

Phylo.write(tree,'new_tree.nwk','newick')
Phylo.draw_ascii(tree)

# Navigate tree and select n random sequences from each clade

In [ ]:
#@markdown ### Set n = number of sequences to select
# If the distance of the parent node is greater than a cutoff value (tree distance/tree leaves) then add genome to list.
# else, keep going down until the distance is greater than the cut off. Add a value
# of genomes calculated as a proportion of genomes in clade and remaining genomes...
# prune processed clades
try:
  from Bio import Phylo
except:
  !pip install biopython 2> /dev/null 1> logs.txt

#@markdown Load tree file in *newick format*
load_tree = True #@param {type:'boolean'}
if load_tree:
  tree_file = '/content/new_tree.nwk' #@param {type:'string'}
  tree = Phylo.read(tree_file,'newick')

#@markdown Set the number of sequences to select
n = 100 #@param {type: 'integer'}
tree.ladderize
tn = tree.count_terminals()
tlen = tree.total_branch_length()
average_len_per_n = tlen/(tn/2)
#genome_files

In [ ]:
#@markdown ### Initialize tree variables
def get_parent_n(tree,n):
  trayectory = tree.get_path(n)
  steps = len(trayectory)
  if steps == 0:
    return(False)
  elif steps == 1:
    return(tree)
  else:
    return(trayectory[steps-2])

i=1
for node in tree.get_nonterminals(order='level'):
  node.name = str(i)
  node.nseqs = 0
  i += 1

for node in tree.get_terminals(order='level'):
  node.nseqs = 0

tree.nseqs = n
tree.name = 'Root'
parent=tree
available_seqs = n
parent_term = tn

In [ ]:
#@markdown ### Select sequences based on nodes structure.
#@markdown If a node has a child that is terminal it will be selected if it distance is less than the average distance per sequence divided by two.
#@markdown Else only one child per node will be selected.
for node in tree.get_nonterminals(order='level'):
  added_seqs = 0
  terminal_not_added = True
  nterm = node.count_terminals()
  #print(node.name, nterm, add_n, available_seqs)
  if available_seqs > 2:
    if len(node) == 2:
      for child in node:
        if child.is_terminal() and child.branch_length < average_len_per_n:
          child.nseqs = 1
          added_seqs += 1
        elif child.is_terminal() and terminal_not_added:
          child.nseqs = 1
          added_seqs += 1
          terminal_not_added = False
  elif available_seqs == 1:
    leave = node.get_terminals()[0]
    leave.nseqs = 1
    added_seqs += 1
    node.nseqs = 0
  available_seqs = available_seqs - added_seqs

In [ ]:
#@markdown ### Show tree
print(f'A total of {sum([node.nseqs for node in tree.get_terminals() if node.nseqs == 1])} sequences out of {tn} were selected. The input n was {n}.')

selected_leaves = []
for n in tree.get_terminals(order='level'):
  if n.nseqs == 1:
    n.color = 'blue'
    selected_leaves.append(n.name)

Phylo.draw(tree)

In [ ]:
#@markdown ###Export selected accession numbers for analysis
import os
selected_leaves_acc = [ l.split('[')[1].split(']')[0] for l in selected_leaves ]
selected_id_list = [ g for g in genome_files if os.path.basename(g).split('.')[0] in selected_leaves_acc]

with open('selected_files.txt','w+') as f:
  for sid in selected_id_list:
    f.write(f'{sid}\n')